In [1]:
from datetime import datetime
import math
import time
import tensorflow as tf

In [2]:
batch_size = 32
num_batches = 100

In [3]:
def print_activation(tensor):
    print(tensor.op.name," ", tensor.get_shape().as_list())

In [4]:
def inference(images):
    parameters = []
    with tf.name_scope('conv1') as scope:
        kernel = tf.Variable(tf.truncated_normal([11,11,3,64], dtype=tf.float32, stddev=1e-1), name='weight')
        conv = tf.nn.conv2d(images, kernel, [1,4,4,1], padding='SAME')
        biases = tf.Variable(tf.constant(0.0,shape=[64],dtype=tf.float32), trainable=True, name='biases')
        bias = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(bias, name=scope)
        print_activation(conv1)
        parameters=parameters+[kernel,biases]
        
    lrn1 = tf.nn.lrn(conv1, depth_radius=4,bias=1.0, alpha=0.001/9, beta=0.75, name='lrn1')
    pool1 = tf.nn.max_pool(lrn1, ksize=[1,3,3,1], strides=[1,2,2,1], padding='VALID', name='pool1')
    print_activation(pool1)
        
    with tf.name_scope('conv2') as scope:
        kernel = tf.Variable(tf.truncated_normal([5,5,64,192], dtype=tf.float32, stddev=1e-1),name='weight')
        conv = tf.nn.conv2d(pool1, kernel, [1,1,1,1], padding='SAME')
        biases = tf.Variable(tf.constant(0.0,shape=[192], dtype=tf.float32), trainable=True, name='biases')
        bias = tf.nn.bias_add(conv, biases)
        conv2 = tf.nn.relu(bias, name=scope)
        parameters=parameters+[kernel,biases]
    
    lrn2=tf.nn.lrn(conv2,depth_radius=4,bias=1.0, alpha=0.001/9,beta=0.75, name='lrn2')
    pool2 = tf.nn.max_pool(lrn2, ksize=[1,3,3,1], strides=[1,2,2,1], padding='VALID', name='pool2')
    print_activation(pool2)
    
    with tf.name_scope('conv3') as scope:
        kernel = tf.Variable(tf.truncated_normal([3,3,192,384],dtype=tf.float32,stddev=1e-1),name='weight')
        conv = tf.nn.conv2d(pool2,kernel,[1,1,1,1], padding='SAME')
        biases = tf.Variable(tf.constant(0.0,shape=[384],dtype=tf.float32),trainable=True, name='biases')
        bias = tf.nn.bias_add(conv, biases)
        conv3 = tf.nn.relu(bias, name=scope)
        parameters=parameters+[kernel,biases]
        print_activation(conv3)
    
    with tf.name_scope('conv4') as scope:
        kernel = tf.Variable(tf.truncated_normal([3,3,384,256], dtype=tf.float32, stddev=1e-1),name='weight')
        conv = tf.nn.conv2d(conv3,kernel,[1,1,1,1], padding='SAME')
        biases = tf.Variable(tf.constant(0.0,shape=[256],dtype=tf.float32), trainable=True, name='biases')
        bias = tf.nn.bias_add(conv,biases)
        conv4 = tf.nn.relu(bias)
        parameters= parameters+[kernel, biases]
        print_activation(conv4)
        
    with tf.name_scope('conv5') as scope:
        kernel = tf.Variable(tf.truncated_normal([3,3,256,256], dtype=tf.float32, stddev=1e-1), name='weight')
        conv = tf.nn.conv2d(conv4, kernel, [1,1,1,1], padding='SAME')
        biases = tf.Variable(tf.constant(0.0,shape=[256], dtype=tf.float32), trainable=True,name='biases')
        bias = tf.nn.bias_add(conv, biases)
        conv5 = tf.nn.relu(bias, name=scope)
        parameters=parameters+[kernel,biases]
        print_activation(conv5)
        
    pool5 = tf.nn.max_pool(conv5, ksize=[1,3,3,1], strides=[1,2,2,1], padding='VALID', name='pool5')
    print_activation(pool5)
    return pool5, parameters

In [5]:
def time_tensorflow_run(session, target, info_string):
    num_steps_burn_in =10
    total_duration = 0.0
    total_duration_squared = 0.0
    
    for i in range(num_batches+num_steps_burn_in):
        start_time = time.time()
        _=session.run(target)
        duration = time.time()-start_time
        if i >= num_steps_burn_in:
            if not i%10:
                #print('%s: step %d, duration = %.3%f' % (datetime.now(), i-num_steps_burn_in,duration))
                print ('%s: step %d, duration = %.3f' %
                       (datetime.now(), i - num_steps_burn_in, duration))
            total_duration=total_duration+duration
            total_duration_squared=total_duration_squared+duration*duration
    mn = total_duration/num_batches
    vr = total_duration_squared/num_batches-mn*mn
    sd = math.sqrt(vr)
    print('%s: %s across %d steps, %.3f +/-%.3f sec / batch' % (datetime.now(),info_string,num_batches,mn,sd))

In [8]:
def run_benchmark():
    with tf.Graph().as_default():
        image_size = 224
        images = tf.Variable(tf.random_normal([batch_size,image_size,image_size,3],dtype=tf.float32,stddev=1e-1))
        pool5, paramters = inference(images)
        init = tf.global_variables_initializer()
        sess = tf.Session()
        sess.run(init)
        
        time_tensorflow_run(sess,pool5,"Forward")
        print(paramters)
        objective = tf.nn.l2_loss(pool5)
        grad = tf.gradients(objective,paramters)
        time_tensorflow_run(sess, grad, "Forward-backward")
        print(paramters)

In [9]:
run_benchmark()

conv1   [32, 56, 56, 64]
pool1   [32, 27, 27, 64]
pool2   [32, 13, 13, 192]
conv3   [32, 13, 13, 384]
conv4/Relu   [32, 13, 13, 256]
conv5   [32, 13, 13, 256]
pool5   [32, 6, 6, 256]
2019-05-18 13:49:50.699957: step 0, duration = 0.300
2019-05-18 13:49:53.505980: step 10, duration = 0.295
2019-05-18 13:49:56.291132: step 20, duration = 0.294
2019-05-18 13:49:59.192201: step 30, duration = 0.290
2019-05-18 13:50:02.094975: step 40, duration = 0.301
2019-05-18 13:50:04.892675: step 50, duration = 0.290
2019-05-18 13:50:07.699899: step 60, duration = 0.300
2019-05-18 13:50:10.513683: step 70, duration = 0.222
2019-05-18 13:50:13.404341: step 80, duration = 0.290
2019-05-18 13:50:16.291967: step 90, duration = 0.295
2019-05-18 13:50:18.811226: Forward across 100 steps, 0.284 +/-0.032 sec / batch
[<tf.Variable 'conv1/weight:0' shape=(11, 11, 3, 64) dtype=float32_ref>, <tf.Variable 'conv1/biases:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'conv2/weight:0' shape=(5, 5, 64, 192) dtype=floa